This notebook contains further analysis and all visualisations used in slides. 

In [9]:
# Import data manipulation libraries
import numpy as np 
import pandas as pd 
# read in data
df = pd.read_excel("Candidate Assignment - Data Analyst 8.18.22.xlsx",sheet_name='Data')
df

,Description,Market,Date,Network,Households,Hours
0,Analyst Test 63000-70000,Rochester-Ny,2018-06-01,WHAM - ABC,11090.0,4554.0
1,Analyst Test 63000-70000,Rochester-Ny,2018-06-01,WHEC - NBC,8037.0,3242.0
2,Analyst Test 63000-70000,Rochester-Ny,2018-06-01,SPECTRUM NEWS - ROCHESTER,7157.0,2601.0
3,Analyst Test 63000-70000,Rochester-Ny,2018-06-01,WROC - CBS,3635.0,1413.0
4,Analyst Test 63000-70000,Rochester-Ny,2018-06-01,FOX NEWS CHANNEL,3216.0,1257.0
...,...,...,...,...,...,...
2767,Analyst Test 90000-93000,Rochester-Ny,2018-08-31,WHAM - ABC,6235.0,2535.0
2768,Analyst Test 90000-93000,Rochester-Ny,2018-08-31,FOX NEWS CHANNEL,3343.0,1336.0
2769,Analyst Test 90000-93000,Rochester-Ny,2018-08-31,MSNBC,2775.0,1114.0
2770,Analyst Test 90000-93000,Rochester-Ny,2018-08-31,CNN,2733.0,1026.0


In [98]:
# I want to create a dataframe that specifies each networks total morning viewership on a given day
# I can do this by creating a column that combines the date, and the network, and then grouping by this column 
df['Date + Network'] = df['Network'] +' : ' + df['Date'].astype(str)

# Sanity check, there should be 66*7 unique items here, (66 days, 7 networks):
len(df['Date + Network'].unique())


462

In [40]:
# Now we get total morning viewership, for a particular day, for a particular network:
morningViewership = df.groupby('Date + Network').agg({'Hours':np.sum}).reset_index()

# I'll re split up date and network for easier visualisation later
morningViewership['Network'] = morningViewership['Date + Network'].str.split(' : ').apply(lambda x: x[0])
morningViewership['Date'] = morningViewership['Date + Network'].str.split(' : ').apply(lambda x: x[1])

# I'll clean up the data, renaming the hours column to be more specic,
# Droping the unesscary column, and changing the date column back to a datetime object
morningViewership = morningViewership.rename(columns={'Hours':'Total Morning Hours'}).drop('Date + Network', axis=1)
morningViewership['Date'] = morningViewership['Date'].apply(pd.to_datetime)
morningViewership


,Total Morning Hours,Network,Date
0,4397.0,CNN,2018-06-01
1,4622.0,CNN,2018-06-04
2,5515.0,CNN,2018-06-05
3,4976.0,CNN,2018-06-06
4,4579.0,CNN,2018-06-07
...,...,...,...
457,8640.0,WROC - CBS,2018-08-27
458,9691.0,WROC - CBS,2018-08-28
459,8550.0,WROC - CBS,2018-08-29
460,8759.0,WROC - CBS,2018-08-30


In [97]:
# Now I want to visualize it as a time series 
from ctypes import sizeof
from turtle import title, width
import altair as alt


# Visualize it
networks = alt.Chart(morningViewership).mark_line(
    point=alt.OverlayMarkDef()
).encode(
    x = 'Date',
    y= 'Total Morning Hours',
    color = 'Network',
).properties(
    width= 900,
    height= 600,
    title = 'Morning Viewership In Rochester - Summer 2018'
)


# Create a line for the launchdate to layer on top:
launchdate = alt.Chart(morningViewership[morningViewership['Date'] == '2018-07-17']).mark_line(
    color='black', tooltip='Launch Date', size = 2,
).encode(
    x = 'Date',
    y= 'Total Morning Hours'
).properties(
    width= 900,
    height= 600,
)
networks + launchdate

alt.LayerChart(...)

In [105]:
# Now I want to remove outliers so I can get a better analysis of average
# Outliers appear on '2018-06-28' and '2018-7-18'
# Boolean mask it:
df1= df[(df['Date'] != '2018-06-28') & (df['Date'] != '2018-7-18')]


# Sanity check, should equal 7*6*2 ( 7 networks, 6 blocks per date, 2 dates)
len(df) - len(df1)

84

In [112]:
# Now i've removed outliers, I'll just copy paste the most of the same code I used above, but this time i'll only look at spectrum
# I'll Only look at spectrum
spectrumShowTimedf = df1[df1['Network'] == 'SPECTRUM NEWS - ROCHESTER']

# I'll remove the 6:30-7:00 block, and the 9:00-9:30 block, because I want to see if the show is doing better than the program that came before it
spectrumShowTimedf = spectrumShowTimedf[(spectrumShowTimedf['Description'] != 'Analyst Test 63000-70000') & ((spectrumShowTimedf['Description'] != 'Analyst Test 90000-93000'))]
spectrumShowTimedf

,Description,Market,Date,Network,Households,Hours,Date + Network
464,Analyst Test 70000-73000,Rochester-Ny,2018-06-01,SPECTRUM NEWS - ROCHESTER,7857.0,2871.0,SPECTRUM NEWS - ROCHESTER : 2018-06-01
471,Analyst Test 70000-73000,Rochester-Ny,2018-06-04,SPECTRUM NEWS - ROCHESTER,8164.0,2963.0,SPECTRUM NEWS - ROCHESTER : 2018-06-04
478,Analyst Test 70000-73000,Rochester-Ny,2018-06-05,SPECTRUM NEWS - ROCHESTER,7686.0,2855.0,SPECTRUM NEWS - ROCHESTER : 2018-06-05
485,Analyst Test 70000-73000,Rochester-Ny,2018-06-06,SPECTRUM NEWS - ROCHESTER,7766.0,2871.0,SPECTRUM NEWS - ROCHESTER : 2018-06-06
492,Analyst Test 70000-73000,Rochester-Ny,2018-06-07,SPECTRUM NEWS - ROCHESTER,7591.0,2794.0,SPECTRUM NEWS - ROCHESTER : 2018-06-07
...,...,...,...,...,...,...,...
2277,Analyst Test 83000-90000,Rochester-Ny,2018-08-27,SPECTRUM NEWS - ROCHESTER,7327.0,2744.0,SPECTRUM NEWS - ROCHESTER : 2018-08-27
2284,Analyst Test 83000-90000,Rochester-Ny,2018-08-28,SPECTRUM NEWS - ROCHESTER,8354.0,3392.0,SPECTRUM NEWS - ROCHESTER : 2018-08-28
2291,Analyst Test 83000-90000,Rochester-Ny,2018-08-29,SPECTRUM NEWS - ROCHESTER,7485.0,2810.0,SPECTRUM NEWS - ROCHESTER : 2018-08-29
2298,Analyst Test 83000-90000,Rochester-Ny,2018-08-30,SPECTRUM NEWS - ROCHESTER,7887.0,2945.0,SPECTRUM NEWS - ROCHESTER : 2018-08-30


In [180]:
# Now I'll do a similar thing as above for visualisation
# Now we get total morning viewership, for a particular day, for a particular network:
spectdf = spectrumShowTimedf.groupby('Date + Network').agg({'Hours':np.sum, 'Households':np.mean,}).reset_index()
# I'll re split up date and network for easier visualisation later
spectdf['Network'] = spectdf['Date + Network'].str.split(' : ').apply(lambda x: x[0])
spectdf['Date'] = spectdf['Date + Network'].str.split(' : ').apply(lambda x: x[1])

# I'll clean up the data, renaming the hours column to be more specic,
# Droping the unesscary column, and changing the date column back to a datetime object
spectdf = spectdf.rename(columns={'Hours':'Total Morning Hours'}).drop('Date + Network', axis=1)
spectdf['Date'] = spectdf['Date'].apply(pd.to_datetime)
spectdf


,Total Morning Hours,Households,Network,Date
0,11909.0,7959.75,SPECTRUM NEWS - ROCHESTER,2018-06-01
1,12061.0,8103.25,SPECTRUM NEWS - ROCHESTER,2018-06-04
2,13302.0,8659.75,SPECTRUM NEWS - ROCHESTER,2018-06-05
3,11730.0,7825.50,SPECTRUM NEWS - ROCHESTER,2018-06-06
4,11383.0,7553.00,SPECTRUM NEWS - ROCHESTER,2018-06-07
...,...,...,...,...
59,10444.0,7102.00,SPECTRUM NEWS - ROCHESTER,2018-08-27
60,12025.0,7899.00,SPECTRUM NEWS - ROCHESTER,2018-08-28
61,10981.0,7438.75,SPECTRUM NEWS - ROCHESTER,2018-08-29
62,11202.0,7622.00,SPECTRUM NEWS - ROCHESTER,2018-08-30


In [178]:
# Visualize it
spectrumSummer = alt.Chart(spectdf).mark_line(
    point=alt.OverlayMarkDef()
).encode(
    alt.Y('Total Morning Hours',
        scale=alt.Scale(domain=(6000,16000))
    ),
    x='Date',

    color = 'Network'
).properties(
    title = 'Spectrum News Total Viewership - 7am-9am - Summer 2018'
)

# add red circle at launch date
launchdate1 = alt.Chart(spectdf[spectdf['Date'] == '2018-07-17']).mark_point(
    color='red', size = 200,
).encode(
    x = 'Date',
    y= 'Total Morning Hours'
)

spectrumSummer + launchdate1

alt.LayerChart(...)

In [201]:
# Next I want to get before and after average viewership to comlement this graph
# So i use the df with only the 7am-9am block for spectrum, with outliers removed

# Before July 17:
beforeLaunchDF = spectdf.set_index('Date')[:'2018-07-16']

#After and including July 17:
afterlaunchDF = spectdf.set_index('Date')['2018-07-17':]

#beforeLaunchDF
afterlaunchDF

,Total Morning Hours,Households,Network
Date,,,
2018-07-17,11213.0,7552.75,SPECTRUM NEWS - ROCHESTER
2018-07-19,10995.0,7447.50,SPECTRUM NEWS - ROCHESTER
2018-07-20,12106.0,7983.00,SPECTRUM NEWS - ROCHESTER
2018-07-23,11137.0,7560.25,SPECTRUM NEWS - ROCHESTER
2018-07-24,11224.0,7597.00,SPECTRUM NEWS - ROCHESTER
2018-07-25,11998.0,8206.00,SPECTRUM NEWS - ROCHESTER
2018-07-26,10842.0,7257.25,SPECTRUM NEWS - ROCHESTER
2018-07-27,12284.0,8043.25,SPECTRUM NEWS - ROCHESTER
2018-07-30,11047.0,7416.25,SPECTRUM NEWS - ROCHESTER


In [204]:
beforeMeanHours = beforeLaunchDF['Total Morning Hours'].mean()
beforeMeanHouseholds = beforeLaunchDF['Households'].mean()
afterMeanHours = afterlaunchDF['Total Morning Hours'].mean()
afterMeanHouseholds = afterlaunchDF['Households'].mean()

print(f'Average total hours before launch: {beforeMeanHours}')
print(f'Average total hours afterlaunch: {round(afterMeanHours)}')

Average total hours before launch: 11397.0
Average total hours afterlaunch: 11236


In [203]:
print(f'Average households watching before launch: {beforeMeanHouseholds}')
print(f'Average households watching after launch: {afterMeanHouseholds}')


Average households watching before launch: 7587.290322580645
Average households watching after launch: 7541.484848484848


In [207]:
# Now i want to a t test
# The samples are differnt sizes so I want independent
from scipy.stats import ttest_ind


ttest_ind(beforeLaunchDF['Total Morning Hours'],afterlaunchDF['Total Morning Hours'])
# We see a huge p-value, meaning these results are insignificant

Ttest_indResult(statistic=0.8420206033178618, pvalue=0.403012721485291)

In [177]:
# Next, I want to look at the pattern in viewership over the course of an average morning
# lets see if the show had any impact on that

df2 = df1[df1['Network'] == 'SPECTRUM NEWS - ROCHESTER']
beforelaunch = df2.set_index('Date')[:'2018-07-16']
afterlaunch = df2.set_index('Date')['2018-07-17':]

# Before launch:
beforeLaunchAVG = beforelaunch.groupby('Description').agg({'Hours':np.mean}).reset_index()
beforeLaunchAVG['Time Block'] = ['6:30-7:00','7:00-7:30','7:30-8:00','8:00-8:30','8:30-9:00','9:00-9:30']

beforechart = alt.Chart(beforeLaunchAVG).mark_line(
    point = alt.OverlayMarkDef()
).encode(
    alt.Y('Hours',
        scale = alt.Scale(zero=False)

    ),
    x='Time Block',
).properties(
    title= 'Average Morning Before Launch of Show'
)


# After Launch:
afterlaunchAVG = afterlaunch.groupby('Description').agg({'Hours':np.mean}).reset_index()
afterlaunchAVG['Time Block'] = ['6:30-7:00','7:00-7:30','7:30-8:00','8:00-8:30','8:30-9:00','9:00-9:30']

afterchart = alt.Chart(afterlaunchAVG).mark_line(
    point = alt.OverlayMarkDef()
).encode(
    alt.Y('Hours',
        scale = alt.Scale(zero=False)

    ),
    x='Time Block'
).properties(
    title='Average Morning After Launch of Show'
)

beforechart | afterchart

alt.HConcatChart(...)